Source:  https://www.kaggle.com/c/fake-news/data#

In [66]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [67]:
df=pd.read_csv("train.csv")

In [68]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [69]:
#drop NaN values

df=df.dropna()

In [70]:
# get independent features
X=df.drop('label',axis=1)

In [71]:
# get dependent features
y=df['label']

In [72]:
X.shape

(18285, 4)

In [73]:
y.shape

(18285,)

In [74]:
import tensorflow as tf

In [75]:
tf.__version__

'2.1.0'

In [76]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [77]:
## vocab size

voc_size=5000

# OneHot representation

In [78]:
messages= X.copy()

In [79]:
messages['title']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799                            What Keeps the F-35 Alive
Name: title, Length: 18285, dtype: object

In [80]:
messages.reset_index(inplace=True)

In [81]:
import nltk
import re
from nltk.corpus import stopwords

In [82]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Harish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# DATA Preprocesssing 

In [83]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [84]:
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [85]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [86]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[4689, 97, 1497, 3866, 3905, 1426, 1561, 4067, 3003, 3690],
 [864, 4266, 2445, 168, 3260, 685, 3624],
 [2591, 991, 2055, 2318],
 [3587, 4230, 2115, 166, 1972, 4557],
 [1594, 3260, 4830, 4116, 3331, 3130, 3260, 264, 1927, 3544],
 [4151,
  230,
  4318,
  4655,
  811,
  4387,
  4798,
  1678,
  371,
  2927,
  972,
  2287,
  3578,
  2272,
  3624],
 [3223, 4696, 1845, 1679, 1485, 87, 3429, 787, 3715, 4376, 2423],
 [2827, 3036, 3132, 4863, 206, 1276, 4387, 4726, 3715, 4376, 2423],
 [3622, 3965, 194, 2853, 756, 1264, 3921, 1934, 4387, 3653],
 [4334, 3883, 4923, 146, 2130, 3508, 2523, 4721],
 [4535, 2103, 2868, 3021, 4797, 2402, 4377, 3857, 4, 352, 73],
 [166, 2253, 3905, 1264, 4387, 206],
 [2461, 475, 430, 2731, 304, 4436, 322, 4844, 204],
 [4454, 2394, 3691, 1727, 4680, 2596, 170, 3715, 4376, 2423],
 [4148, 3529, 434, 825, 2820, 3715, 4376, 2423],
 [3515, 1145, 535, 3166, 717, 2689, 1910, 2133, 1732, 3478],
 [2861, 1995, 4266],
 [4450, 1449, 2973, 4693, 4387, 818, 4148, 3624],
 [586, 3482, 2

# Embedding representation

In [87]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr, padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4067 3003 3690]
 [   0    0    0 ... 3260  685 3624]
 [   0    0    0 ...  991 2055 2318]
 ...
 [   0    0    0 ... 3715 4376 2423]
 [   0    0    0 ... 4852 1503  642]
 [   0    0    0 ... 2780 1843 1609]]


In [88]:
len(embedded_docs)

18285

In [89]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4689,
         97, 1497, 3866, 3905, 1426, 1561, 4067, 3003, 3690])

In [90]:
## creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [91]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [92]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [94]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.1,random_state=42)

In [95]:
X_train.shape

(16456, 20)

In [96]:
X_test.shape

(1829, 20)

In [97]:
y_train.shape

(16456,)

In [98]:
y_test.shape

(1829,)

# Model Training

In [99]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10, batch_size=64)

Train on 16456 samples, validate on 1829 samples
Epoch 1/10
16456/16456 [==============================] - 6s 392us/sample - loss: 0.2917 - accuracy: 0.8652 - val_loss: 0.2072 - val_accuracy: 0.9081
Epoch 2/10
16456/16456 [==============================] - 4s 269us/sample - loss: 0.1377 - accuracy: 0.9461 - val_loss: 0.2093 - val_accuracy: 0.9081
Epoch 3/10
16456/16456 [==============================] - 5s 290us/sample - loss: 0.0986 - accuracy: 0.9620 - val_loss: 0.2401 - val_accuracy: 0.9153
Epoch 4/10
16456/16456 [==============================] - 5s 287us/sample - loss: 0.0698 - accuracy: 0.9745 - val_loss: 0.2599 - val_accuracy: 0.9103
Epoch 5/10
16456/16456 [==============================] - 5s 296us/sample - loss: 0.0491 - accuracy: 0.9837 - val_loss: 0.3025 - val_accuracy: 0.9136
Epoch 6/10
16456/16456 [==============================] - 5s 306us/sample - loss: 0.0348 - accuracy: 0.9898 - val_loss: 0.4162 - val_accuracy: 0.9103
Epoch 7/10
16456/16456 [===========================

# Performance Metrics and accuracy

In [117]:
y_pred=model.predict_classes(X_test)

In [118]:
from sklearn.metrics import confusion_matrix

In [119]:
confusion_matrix(y_test,y_pred)

array([[931, 105],
       [ 70, 723]], dtype=int64)

In [120]:
from sklearn.metrics import accuracy_score

In [121]:
accuracy_score(y_test,y_pred)

0.9043193001640241

# ON TEST dataset

In [122]:
test=pd.read_csv('test.csv')

In [123]:
test.shape

(5200, 4)

In [124]:
Test_id=test["id"]

In [125]:
test=test.drop(['text','id','author'],axis=1)

In [126]:
test.head()

,title
0,"Specter of Trump Loosens Tongues, if Not Purse..."
1,Russian warships ready to strike terrorists ne...
2,#NoDAPL: Native American Leaders Vow to Stay A...
3,"Tim Tebow Will Attempt Another Comeback, This ..."
4,Keiser Report: Meme Wars (E995)


In [127]:
test.isnull().sum()

title    122
dtype: int64

In [128]:
test.fillna('fake',inplace=True)

In [129]:
test.shape

(5200, 1)

In [130]:
testmessage= test.copy()

In [131]:
testmessage.shape

(5200, 1)

In [132]:
corpus_test=[]
for i in range(0,len(testmessage)):
    review=re.sub('[^a-zA-Z]',' ',testmessage['title'][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus_test.append(review)

In [133]:
onehot_repr_test=[one_hot(words,voc_size) for words in corpus_test]
onehot_repr_test

[[4615, 4387, 4421, 3145, 1231, 2342, 4116, 1370, 3715, 4376, 2423],
 [4535, 2897, 3203, 3224, 457, 4284, 1583],
 [2525, 2852, 3198, 2596, 2525, 4364, 4800, 4177, 1548, 237],
 [3609, 2363, 3310, 230, 17, 2423, 388, 3715, 4376, 2423],
 [2726, 2285, 3169, 2051, 3199],
 [4387, 969, 1911, 1070, 2445, 4272, 617],
 [2890, 2296, 2861, 624, 4680, 4535, 4693, 4387, 3624],
 [3852, 1269, 1712, 3033, 4346],
 [419, 4045, 3738, 306, 4060, 3984, 3740],
 [3297],
 [4469, 4464, 3655, 154, 4387, 1751, 434],
 [1201, 81, 1070, 1657, 4143],
 [4473, 4999, 1143, 818, 502, 890, 4953],
 [47, 3382, 2804, 4427],
 [1654, 2971, 2525, 1720, 3251, 475, 2228, 722, 3119, 4387, 3046, 969, 868],
 [804, 621, 1329, 3984, 1013, 1508, 1248, 3715, 4376, 2423],
 [3715,
  3542,
  2329,
  410,
  256,
  1623,
  67,
  1843,
  4824,
  2090,
  97,
  3715,
  4376,
  2423],
 [1280, 1194, 4355, 2122, 3963, 4480, 3108, 810, 3624],
 [4387, 1940, 2657, 166, 688, 1018, 3172, 520, 2445],
 [2988, 1598, 4628, 3173, 2854, 1661, 4437, 2596, 371

In [135]:
sent_length=20
embedded_docs_test=pad_sequences(onehot_repr_test, padding='pre',maxlen=sent_length)
print(embedded_docs_test)

[[   0    0    0 ... 3715 4376 2423]
 [   0    0    0 ...  457 4284 1583]
 [   0    0    0 ... 4177 1548  237]
 ...
 [   0    0    0 ... 3715 4376 2423]
 [   0    0    0 ... 4535 2846 3410]
 [   0    0    0 ... 3715 4376 2423]]


In [136]:
x_test=np.array(embedded_docs_test)

In [137]:
testpredict=model.predict_classes(x_test)

In [138]:
testpredict

array([[0],
       [1],
       [1],
       ...,
       [0],
       [1],
       [0]])

In [139]:
val=[]
for i in testpredict :
    val.append(i[0])

In [140]:
submission = pd.DataFrame({'id':Test_id, 'label':val})
submission.shape

(5200, 2)

In [142]:
submission.to_csv('submission.csv',index=False)